## 모두를 위한 딥러닝 #5

이번 강의에서는 CNN, FC만큼 중요한 가장 많이 쓰이는 RNN이라는 구조를 살펴볼 것이다. RNN은 차원에 시간이 있는 경우에 가장 많이 쓰이는 구조로 
주로 음성 인식, 자연어 처리 등에 가장 많이 사용된다. 주식 데이터의 경우에도 시간에 관련된 데이터이므로 RNN을 적용할 수 있다.

### 학습 순서
- RNN 구조 살펴보기 
- RNN numpy, tensorflow 코드 돌려보기 
- RNN에서 파생된 다른 network 구조 
    - LSTM
    - GRU

### 시간?
아주 간단하게 생각하면 FC에 시간에 관련된 차원이 추가되었다고 보면 된다. 그럼 이를 어떻게 구상할지 아이디어를 떠올려보자.
1. 과거의 정보들을 모아서 미래 정보를 예측하면 어떨까? (노래를 들을 때 0:00 ~ 0:10초까지 들으면 0:11초가 예상이 된다.)
2. 가까운 시간의 정보가 먼 시간의 정보보다 중요할까? (노래의 0:11초를 예측할 때 0:01초보다 0:10초가 더 중요한 경우도 있다. 주로 너무 먼 시간의 정보는 중요하지 않게된다.) <br>

&rarr; 이 내용을 합쳐서 구조를 만들면 아마도 과거의 정보들에 대한 각각의 가중치를 학습해서 현재 정보에 전달해주면 미래를 예측할 수 있겠다.

### Vanilla RNN
Feature들로 이루어진 정보를 전달하며 가중치를 학습하는 방법 Fully Connected Network(FC)을 앞에서 배웠다. 이를 활용해서 시간에 관한 내용도 표현해보자.

왼쪽 그림은 오른쪽을 요약해서 나타낸 것이므로 일단 오른쪽 그림을 보자. <br>

![unrolled_RNN2](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

먼저 항들에 대해서 소개하면 $x$는 input, $s$는 hidden state, $o$는 output이다. 

input의 경우에는 시간에 관련한 차원을 고려했을 때 $x_t$로 표현이 가능하다. 예를 들면, 집 값을 예측하기 위한 feature가 20개인 데이터 $x$가 100개 있다고 가정하면 기존에는 `x.shape = [100, 20]`이 되었는데 여기에 시간에 관한 $t$ 항을 추가해서 **10년 간** 연도 별 데이터를 추가하게 되면 `x.shape = [100, 10, 20]`이 된다. (**되게 헷갈리고 중요하다**)

자 그럼, 위에서 간단한 직관을 통해 본 내용을 적용시켜보자. 데이터는 가장 과거의 input 값으로부터 시간 순으로 정보가 전달된다. 그리고 각 시간 별 데이터에 대해서는 output이 존재한다. (**과거에도 집 값은 있었고 현재에도 집 값은 있다** = $o_t$) 이 때, 모든 연산은 FC로 진행된다. 

위 내용을 나타내는 구조가 바로 오른쪽 그림이다. 

좀 더 세부적인 내용으로 들어가보자. 

#### 1) weight는 아래와 같이 용도를 나눠서 사용한다. 
1. input 값을 다룰 때 사용하는 값($U$) 
2. 시간을 흘러서 전달되는 정보를 다룰 때 사용하는 값($W$)
3. output을 내보낼 때 사용하는 값($V$)

용도가 나뉘는 이유는 FC를 생각해보면 $x_t$, $o_t$, $s_t$의 shape가 모두 다를 것임을 알 수 있다. 
$x_t$는 feature를 포함한 중간 크기의 shape, $s_t$는 정보를 얼마나 크게 표현할지에 대해 결정하므로 큰 shape, $o_t$는 출력 값으로 $x_t$와 같거나 작은 shape을 갖게 될 것이다. 


그럼, 시간에 대해서 weight를 $U, W, V$로 sharing 하는데 이것은 왜 그럴까? 매 시간에 대해서 모든 weight를 나눠서 학습하게 되면 너무 많은 데이터도 필요하고 weight가 잡아먹는 메모리도 엄청나게 된다. 또, 우리가 입력해주는 배열의 크기($t$)도 일정하지 않기 때문에 모든 시간 별로 weight를 학습시키기 어렵기 때문이다.

#### 2) 학습은 Backpropagation을 사용한다. 그래서 현재에 가까운 과거 정보의 영향력이 크다.
FC와 동일한 Backpropagation(*훨씬 졸라 복잡*)을 사용해서 $U, W, V$를 모두 학습시킨다. 근데, Backpropagation는 vanishing gradient의 문제가 있으므로 현재 시간과 먼 과거의 정보일 수록 그 영향력이 약해질 수 밖에 없다. 

근데, 노래를 듣다보면 맨 앞 구절을 듣다가 알아채는 경우가 많다. 다른 경우들에 대해서도 현재에 가까운 정보가 중요하긴 해도 조금 먼 과거 정보도 중요한 경우가 많다. (**문제점**) 이러한 구조는 데이터를 올바르게 해석하는 데에 문제가 되므로 이를 해결하기 위한 **다른 구조(LSTM, GRU)**가 나오게 된다.

#### 3) Activation function은 tanh,  ReLU를 주로 사용한다. 
Backpropagation의 문제를 조금 완화하기 위해서 ReLU를 사용해도 되고 주로 tanh를 많이 사용하는데 sigmoid 대비 backpropagation할 때에도 미분 값이 0~1(sigmoid: 0~0.25)로 크고 망이 깊어질 때 더 빠른 수렴을 하기 때문이라고 한다. 그리고 뒤에서 볼 예정이지만 LSTM에서는 backpropagation의 문제를 어느 정도 해결해서 tanh, sigmoid로만 구성이 된다.

#### 4) 과거의 정보는 다음 $t$의 hidden state에 누적되어 전달된다는 의미?
정보 손실이 없다고 가정하면 현재(마지막 $t$)는 과거의 모든 정보를 누적해서 가지고 있는 모습이 된다. <br>
즉, 노래를 예로 들면 3분 짜리 노래의 모든 정보를 누적해서 가질 수 있다는 것이다! 그럼 누적한 정보를 토대로 노래 장르/노래 이름을 label로 사용해서 학습시킬 수 있다. <br>
다른 예시로 주식 시장을 보자. 10년 전 데이터부터 현재까지의 정보를 누적해서 미래를 예측한다고 하면..? 말이 안 되는게 정보를 누적한들 예측해야 할 미래에 대한 label이 없는데 어떻게 학습할까? 그럼, 30일 단위로 쪼개서 미래를 예측하는 방법을 생각해볼 수 있겠다. 데이터를 잘라서 30일을 지켜본 후 다음 주가 변동 예측 이런 식으로 하면 regression과 같이 오늘 정보를 토대로 내일을 예측하는 것이 아니라 장의 30일 정보를 토대로 내일을 예측하는 것이기 때문에 더 많은 정보를 토대로 예측을 할 수 있겠다. 

아래 코드를 통해서 RNN이 어떤 방식으로 작동하는지 살펴보도록 하자.

### Numpy 예시

In [2]:
import numpy as np

In [153]:
# RNN이 어떤 식으로 작동하는지 확인해보기 위해서 toy code를 만들어보자.
U = np.array([[1, 2, 3, 1],
              [2, 1, 1, 0],
              [1, 3, 0, 1]])

W = np.array([[1, 2, 1, 0],
              [2, 1, 0, 1],
              [0, 1, 2, 2],
              [2, 2, 1, 1]])

V = np.array([[0],
              [1],
              [1],
              [2]])

b_s = np.ones([1, U.shape[1]])
b_o = np.ones([1, V.shape[1]])
시

# X.shape = [batch_size, # of time_step, features]
X = np.array([[[2, 3, 3],
               [1, 2, 1],
               [1, 2, 1],
               [0, 0, 1]],
              [[1, 0, 1],
               [2, 1, 2],
               [3, 0, 1],
               [1, 1, 1]]])
print('X: {}, U: {}, W: {}, V: {}'.format(X.shape, U.shape, W.shape, V.shape))

X: (2, 4, 3), U: (3, 4), W: (4, 4), V: (4, 1)


In [194]:
# 위 내용을 풀어서 전개해보자. x는 2개의 데이터, 4개의 time step, 3개의 feature를 갖는다. 
x0 = X[:, 0, :]
s0 = x0.dot(U) + b_s
o0 = s0.dot(V) + b_o

x1 = X[:, 1, :]
s1 = x1.dot(U) + s0 + b_s
o1 = s1.dot(V) + b_o

x2 = X[:, 2, :]
s2 = x2.dot(U) + s1 + b_s
o2 = s2.dot(V) + b_o

x3 = X[:, 3, :]
s3 = x3.dot(U) + s2 + b_s
o3 = s3.dot(V) + b_o

# 개별 element 들을 하나로 다 합쳐보자.
S = np.stack([s0, s1, s2, s3], axis=1).reshape([s0.shape[0], -1, s0.shape[1]])
O = np.stack([o0, o1, o2, o3], axis=-1).reshape([o0.shape[0], -1, o0.shape[1]])

print('X: {}, S: {}, O: {}'.format(X.shape, S.shape, O.shape))

X: (2, 4, 3), S: (2, 4, 4), O: (2, 4, 1)


확인해보자.

In [195]:
s3

array([[ 28.,  37.,  23.,  14.],
       [ 20.,  35.,  27.,  16.]])

In [197]:
S[:, 3, :]

array([[ 28.,  37.,  23.,  14.],
       [ 20.,  35.,  27.,  16.]])

In [198]:
o3

array([[ 89.],
       [ 95.]])

In [199]:
O[:, 3, :]

array([[ 89.],
       [ 95.]])

In [207]:
# 위 내용을 loop를 좀 더 정리해서 깔끔하게 나타내면
time_step = 4
batch_size = 2
num_features = 3
s_prev = np.zeros([2, 4])
def forward_rnn(inputs, s_prev):
    x, s, o = {}, {}, {}
    s[-1] = s_prev
    for t in range(time_step):
        x[t] = X[:, t, :]
        s[t] = x[t].dot(U) + s[t - 1] + b_s
        o[t] = s[t].dot(V) + b_o
    S = np.stack([s[i] for i in range(0, time_step)], axis=1).reshape([s0.shape[0], -1, s0.shape[1]])
    O = np.stack([o[i] for i in range(0, time_step)], axis=-1).reshape([o0.shape[0], -1, o0.shape[1]])
    return S, O

In [201]:
o[3]

array([[ 89.],
       [ 95.]])

In [202]:
s[3]

array([[ 28.,  37.,  23.,  14.],
       [ 20.,  35.,  27.,  16.]])

In [209]:
S, O = forward_rnn(X, s_prev)

In [210]:
S[:, 3, :]

array([[ 28.,  37.,  23.,  14.],
       [ 20.,  35.,  27.,  16.]])

In [211]:
O[:, 3, :]

array([[ 89.],
       [ 95.]])

### tensorflow 예시

- 위에서 output에 사용된 `V`항이 없어서 필요 시 따로 붙여줘야 한다. (사실 우리의 목적은 과거의 정보를 현재까지 모으는 것이기 때문에 각 시간 별 output을 쓸 일이 거의 없음.)
- tensorflow 내부 구현되어 있는 함수들을 사용하면 $S = [s_0, s_1, ...s_t]$에 대한 출력 값을 얻을 수 있다.

In [91]:
import tensorflow as tf

In [107]:
tf.reset_default_graph()

X_tf = tf.placeholder(tf.float32, [None, 4, 3])
U_tf = tf.constant(U)
V_tf = tf.constant(V)

cell = tf.contrib.rnn.BasicRNNCell(num_units=4)
output, states = tf.nn.dynamic_rnn(cell=cell, inputs=X_tf, initial_state=None, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_val, states_val = sess.run([output, states], feed_dict={X_tf: X})

In [108]:
output_val.shape

(2, 4, 4)

In [109]:
states_val.shape

(2, 4)

In [105]:
O

array([[[ 40.],
        [ 17.],
        [ 60.],
        [ 47.]],

       [[ 80.],
        [ 77.],
        [ 89.],
        [ 95.]]])

### LSTM, GRU

위에서 vanilla RNN은 큰 문제가 있기 때문에 다른 구조가 생겨났다고 소개했다. 그 중에 LSTM에 대해서 살펴보도록 하자. (GRU는 LSTM에서 파생된 구조라 거의 비슷하다.)

아래는 전체적인 구조이다. Vanilla RNN으로 생각했을 때 1개 step에 해당되는 모습이며 비교해보면 훨씬 복잡한 것을 알 수 있다. 그리고 이 구조를 설명할 때에는 위에서 봤을 때의 $o$에 해당되는 항은 생략하고 진행하도록 한다. $o$를 구하고 싶으면 $h$에 network 하나를 추가해주면 된다.

![LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)
![LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM2-notation.png)

**1. Step 1: forget gate** <br>

새롭게 등장한 **cell state($c_t$)**라는 구성요소에 대해 먼저 소개하면 각 $t$의 cell에서의 정보를 저장한다고 생각하면 되겠다. 다음 cell에 내보낼 정보도 가지고 있고 hidden state로 내보낼 정보도 가지고 있다. (*물론 얼마나 내보내냐에 대한 로직이 구조 내에 있는 연산자들이다.*)

LSTM cell(RNN의 기본단위)도 기본적으로 input $x_t$를 받고 이전 time step에서의 hidden state $h_{t-1}$를 받는다. (위에서의 표기와 변경된 표기: $s \rightarrow h$) <br> 그리고 forget gate에서는 vanilla RNN에서와 다르게 $x_t, h_{t-1}$를 concatenate로 합친다. (*이전에는 $x_t$를 network 통과시킨 뒤 add*) 최종적으로 얻은 값을 sigmoid를 통과시켜서 0~1의 값으로 구한다. 이 값을 $f_t$라고 하겠다. 

$f_t$가 하는 일은 cell_state에서 넘어온 정보를 현재의 정보를 바탕으로 이 후에 얼마나 전달해야 할 지를 결정한다. 즉, 이전 cell의 정보인 $c_{t-1}$가 현재 cell로 넘어오게 되는데 이 중에 얼만큼을 반영해서 현재 $c_t$을 만들어줄 것인지를 결정해야 한다. 얼마나 전달해야 하는지 결정한다는 의미는 얼마나 까먹어야 할 지로 생각할 수 있으므로 **forget gate**라고 부른다.
![LSTM step1](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)

**2. Step 2: input gate, candidate value**

각 cell에는 input $x$와 이전 cell의 hidden state $h$가 전달된다. 이 정보들을 조합해서 얼마나 입력해줄지 결정해 줘야 한다. 얼마나 입력해줄 지에 대한 내용은 위에서 본 sigmoid로 결정할 수 있고 0~1의 값으로 결정하면 되겠다. 이를 **input gate** $i_t$라고 한다. 

$\tilde C_t$는 candidate value로 볼 수 있는데 이는 두 정보가 들어왔으니 이를 조합한 하나의 정보로 만들어주는 것이다. 이 정보에 input gate에서의 값이 곱해져서 cell로 얼마나 넘어갈 것인지가 결정이 된다. 

여기까지 보면 이전 cell의 값을 얼마나 전달해줄 것인지 결정한 뒤에 input 값의 조합을 얼마나 cell에 전달해줄 지 결정을 한 것이다. 

![LSTM step2](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)

**3. Step 3: output(hidden state)**

step 2까지의 과정으로 cell에 충분한 정보가 모인 것을 확인했다. 이제 다음 hidden state인 $h_t$로 얼마의 정보를 넘겨줘야 할 지 결정해야 한다. 이 때 **값 자체는 cell인 $C_{t}$로부터** 가져오지만 **얼마나**에 대한 측면은 input 값으로부터 결정된다. 직관적으로 생각했을 때 hidden state를 결정한다면 가장 영향을 많이 주는 값은 이전 hidden state(***지금 hidden이 input을 제외하고 많은 비중을 두고 참조하고 있는 값***)
와 새로운 input이 될 것이다. 그래서 둘을 고려해서 다음 hidden state에 대한 비율을 결정한다.

![LSTM step3](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)

#### 해결 가능한 문제점

1) Vanishing gradient 

$C_t$라는 cell **정보를 담당**하는 항이 생겼고 이는 직선적으로 다른 step 들과 연결된다. 그 뜻은, backpropagation을 할 때 step 별로 vanishing gradient 없이 $C_t$에 대한 영향력이 그대로 전달될 수 있다는 뜻이 된다. hidden_state가 가장 중요한 값인데 어차피 $C_t$가 vanishing gradient 이슈가 없으면 $h_t = o_t \cdot \tanh(C_t)$만 계산하면 간단하게 hidden_state가 구해져서 마찬가지로 vanishing gradient가 없다고 볼 수 있다.

Vanishing gradient을 해결하면 긴 time step을 갖는 데이터 (예를 들면 노래 전곡, 주식 데이터 1달 단위...)를 다룰 수 있다는 뜻이다.

#### 아직도 해결 해야 할 문제점 

**1) 정보 전달의 방향성**

과거 &rarr; 현재로 시간적 이동이 있는 경우도 있지만, 노래처럼 뒷 부분을 들어도 앞 부분을 알 수 있는 경우가 있다. 이런 경우에는 앞 부분의 정보가 뒷 부분보다 중요하다고 볼 수 없다. 그럼, 양 방향 모두 고려해줄 순 없을까? &rarr; **Bi-directional RNN**

양 방향으로 고려하는 것 외에도 RNN layer도 다른 layer들처럼 쌓을 수 있다.

**2) 압축된 정보를 푸는 방법**

time-series 데이터의 정보를 압축시킨 뒤 그 정보를 토대로 노래 제목이나 주식 가격을 예측하는 문제는 쉽게 접근할 수 있다. 하지만, 압축한 뒤에 풀어야 하는 경우가 생긴다면? 정보를 변환하는 경우(번역이나 음성 인식 등)가 이에 해당된다. 음.. RNN을 통과시켜서 압축된 정보를 다시 RNN을 통과시켜서 다른 정보로 변환할 수는 없을까? &rarr; encoder-decoder(**Seq2Seq**)

실제로 Seq2Seq 모델 구조는 번역이나 음성 인식 쪽에 많이 사용되고 있으며 거의 전통적인 머신러닝 방법들을 대거 교체하고 있다.

Reference: <br>
[colah의 블로그 - LSTM 설명 자료](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) <br>
[RNN 설명 이미지](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

애매한 내용들: <br>
[LSTM에서 ReLU를 사용하지 않는 이유](https://www.reddit.com/r/MachineLearning/comments/2t1rsp/lstm_question/) <br>
[tanh vs sigmoid 사용](https://stats.stackexchange.com/questions/142348/tanh-vs-sigmoid-in-neural-net_)